Importovanie knižníc, funkcie na úpravu dátumu a nastavenie adresy pre ElasticSearch

In [133]:
from bs4 import BeautifulSoup
import re
import json
import calendar
import glob
from elasticsearch import Elasticsearch
from unidecode import unidecode
import datetime
abbr_to_num = {name: num for num, name in enumerate(calendar.month_abbr) if num}
es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

Funkcia na extrahovanie informácií z HTML stánky spolu s ošetrením, ak sa daný údaj na stránke nenachádza.

In [134]:
def parse_html(html):
#     print(html)
#     try
#     name = re.search('(?<=(Original Title<\/div>\n)).*(?=\ \<)',html)
    try:
        ident = re.search('\/book\/show\/([0-9]*)',html).group(1)
    except: 
        ident = None
    try:
        description = re.search('id="description"(.*\n){3}.*"\>(.*)\<\/span',html).group(2)
    except: 
        description = None
    try:
        description = description.replace('<br /><br />', ' ')
    except: 
        description = None
    try:
        name = re.search('\<h1\ id="bookTitle.*\n\s*(.*)',html).group(1)
    except: 
        name = None
    try:
        isbn = re.search('(?<=(ISBN<\/div>\n)).*\n\s*(.*)',html).group(2)
    except: 
        isbn = None
    try:
        isbn13 = re.search('(?<=(ISBN<\/div>\n)).*\n.*\n.*((?<=\>).*)(?=\<span)[^0-9]*([0-9]+)',html).group(3)
    except: 
        isbn13 = None
    try:
        series = re.search('(?<=(Series<\/div>\n))((\s*\<d|\s*\<a).*\n){0,10}\s*<\/div>',html).group()
    except: 
        series = None
    try:
        series = re.findall('\>([a-zA-Z0-9#\ ]*)\<\/a',series)
    except: 
        series = None
    try:
        characters = re.findall('\/characters.{1,40}"\>([a-zA-z0-9\ ]*)\<',html)
    except: 
        characters = None
        
    create_json(ident, name, isbn, isbn13, series, description, characters)

Funkcia zo vstupných údajov vytvorí JSON formát, a vlozi ho do elastic search databazy.

In [135]:
def create_json(ident, name, isbn, isbn13, series, description, characters):
    x = {
      "id": ident,
      "name": name,
      "isbn": isbn,
      "isbn13": isbn13,
      "series": series,
      "description": description,
      "characters": characters
    }
    y = json.dumps(x)
    global es
    page_id=0
    page_id += 1
    print(json.dumps(json.loads(y), indent=2))
    es.index(index='steam', id=page_id, body=y)

Spracovanie všetkých HMTL súborov uložených v adresári.

In [136]:
pages = glob.glob("D:/School/VINF/Data/*.html")
for page in pages:
    html = open(page,encoding="utf8")    
    soup = html.read()
#     print(soup)
    parse_html(soup)

{
  "id": "100915",
  "name": "The Lion, the Witch and the Wardrobe",
  "isbn": null,
  "isbn13": null,
  "series": [],
  "description": "Narnia...the land beyond the wardrobe door, a secret place frozen in eternal winter, a magical country waiting to be set free. Lucy is the first to find the secret of the wardrobe in the professor's mysterious old house. At first her brothers and sister don't believe her when she tells of her visit to the land of Narnia. But soon Edmund, then Peter and Susan step through the wardrobe themselves. In Narnia they find a country buried under the evil enchantment of the White Witch. When they meet the Lion Aslan, they realize they've been called to a great adventure and bravely join the battle to free Narnia from the Witch's sinister spell.",
  "characters": [
    "Aslan",
    "Lucy Pevensie",
    "Edmund Pevensie",
    "Peter Pevensie",
    "Susan Pevensie",
    "Jadis"
  ]
}
{
  "id": "10644631",
  "name": "The Eye of the World: The Graphic Novel, Volum

{
  "id": "13521459",
  "name": "Red Country",
  "isbn": "0316214442",
  "isbn13": "9780316214445",
  "series": [
    "First Law World #6"
  ],
  "description": "They burned her home.<br />They stole her brother and sister.<br />But vengeance is following. Shy South hoped to bury her bloody past and ride away smiling, but she'll have to sharpen up some bad old ways to get her family back, and she's not a woman to flinch from what needs doing. She sets off in pursuit with only a pair of oxen and her cowardly old step father Lamb for company. But it turns out Lamb's buried a bloody past of his own. And out in the lawless Far Country the past never stays buried. Their journey will take them across the barren plains to a frontier town gripped by gold fever, through feud, duel and massacre, high into the unmapped mountains to a reckoning with the Ghosts. Even worse, it will force them into an alliance with Nicomo Cosca, infamous soldier of fortune, and his feckless lawyer Temple, two men no

{
  "id": "17332218",
  "name": "Words of Radiance",
  "isbn": "0765326361",
  "isbn13": "9780765326362",
  "series": [
    "The Stormlight Archive #2",
    "The Cosmere"
  ],
  "description": "<b><i>Words of Radiance</i>, Book Two of the Stormlight Archive, continues the immersive fantasy epic that <i>The Way of Kings</i> began.</b> Expected by his enemies to die the miserable death of a military slave, Kaladin survived to be given command of the royal bodyguards, a controversial first for a low-status \"darkeyes.\" Now he must protect the king and Dalinar from every common peril as well as the distinctly uncommon threat of the Assassin, all while secretly struggling to master remarkable new powers that are somehow linked to his honorspren, Syl. The Assassin, Szeth, is active again, murdering rulers all over the world of Roshar, using his baffling powers to thwart every bodyguard and elude all pursuers. Among his prime targets is Highprince Dalinar, widely considered the power behind 

{
  "id": "27264916",
  "name": "White Collar Blackmail",
  "isbn": null,
  "isbn13": null,
  "series": null,
  "description": " Directors and executives of public corporations are dying in mysterious circumstances. <p>.Their only connection is irregular share dealings that took place in their companies before their deaths.</p><p>When young auditor, Todd Hansen, runs up a huge gambling debt with illegal bookies, he finds himself in serious trouble. The only way he can pay them is by providing confidential information about his firm\u2019s corporate clients.</p><p>Caught, and sentenced to nine years in New York\u2019s toughest prison, Todd is given a way out BUT it\u2019s dangerous. There are those who think they\u2019d be safer if he was dead.</p> Will a chance meeting with a Mafia boss in prison save Todd? <p>\u00a0</p>",
  "characters": []
}
{
  "id": "2767793",
  "name": "The Hero of Ages",
  "isbn": "0765316897",
  "isbn13": "9780765316899",
  "series": [
    "Mistborn #3",
    "Th

{
  "id": "34002132",
  "name": "Oathbringer",
  "isbn": "076532637X",
  "isbn13": "9780765326379",
  "series": [
    "The Stormlight Archive #3",
    "The Cosmere"
  ],
  "description": "In <i>Oathbringer</i>, the third volume of the <i>New York Times</i> bestselling Stormlight Archive, humanity faces a new Desolation with the return of the Voidbringers, a foe with numbers as great as their thirst for vengeance. Dalinar Kholin's Alethi armies won a fleeting victory at a terrible cost: The enemy Parshendi summoned the violent Everstorm, which now sweeps the world with destruction, and in its passing awakens the once peaceful and subservient parshmen to the horror of their millennia-long enslavement by humans. While on a desperate flight to warn his family of the threat, Kaladin Stormblessed must come to grips with the fact that the newly kindled anger of the parshmen may be wholly justified. Nestled in the mountains high above the storms, in the tower city of Urithiru, Shallan Davar in

{
  "id": "41865",
  "name": "Twilight",
  "isbn": "0316015849",
  "isbn13": "9780316015844",
  "series": [
    "The Twilight Saga #1"
  ],
  "description": null,
  "characters": [
    "Edward Cullen",
    "Jacob Black",
    "Laurent",
    "Renee",
    "Bella Swan",
    "Billy Black",
    "Esme Cullen",
    "Alice Cullen",
    "Jasper Hale",
    "Carlisle Cullen",
    "Emmett Cullen",
    "Rosalie Hale",
    "Charlie Swan",
    "Mike Newton",
    "Jessica Stanley",
    "Angela Weber",
    "Tyler Crowley"
  ]
}
{
  "id": "41952489",
  "name": "The Rage of Dragons",
  "isbn": "031648976X",
  "isbn13": "9780316489768",
  "series": [
    "The Burning #1"
  ],
  "description": "The Omehi people have been fighting an unwinnable fight for almost two hundred years. Their society has been built around war and only war. The lucky ones are born gifted. One in every two thousand women has the power to call down dragons. One in every hundred men is able to magically transform himself into a bigger,

{
  "id": "52233178",
  "name": "The Octunnumi Fosbit Files Prologue",
  "isbn": null,
  "isbn13": null,
  "series": null,
  "description": "The Octunnumi does not exist. Fosbit and any files relating to a Fosbit do not exist. The Tarelen peoples that protect and provide sanctuary for the Avitens of Fethrist are not real. The abilities of the Tarelen peoples with their heightened skills, living in their Utopian world are myth, their purpose here mere tales; rumours of their reincarnations enabling them to live many lives\u2026 ridiculous! And there is definitely no magic. None at all. No, really, there isn\u2019t. Magic is not a thing. There are also no hidden worlds or mythical beings coexisting just out of sight.  Equally, there are no secret access points to these hidden worlds that don\u2019t exist, and there is no, 'unfinished business from the past' that is set to destroy, well, anything.  There is no disaster looming. Anyway, regardless of any potential threat that may or may no

{
  "id": "9579634",
  "name": "Prince of Thorns",
  "isbn": "0007423292",
  "isbn13": "9780007423293",
  "series": [
    "The Broken Empire #1"
  ],
  "description": "Before the thorns taught me their sharp lessons and bled weakness from me I had but one brother, and I loved him well. But those days are gone and what is left of them lies in my mother's tomb. Now I have many brothers, quick with knife and sword, and as evil as you please. We ride this broken empire and loot its corpse. They say these are violent times, the end of days when the dead roam and monsters haunt the night. All that's true enough, but there's something worse out there, in the dark. Much worse. From being a privileged royal child, raised by a loving mother, Jorg Ancrath has become the Prince of Thorns, a charming, immoral boy leading a grim band of outlaws in a series of raids and atrocities. The world is in chaos: violence is rife, nightmares everywhere. Jorg has the ability to master the living and the dead, 